### 筆記
* annotation version: tumor as kidney(DATA_0811_2)
* INPUT_SIZE = 512
* EPOCHS = 100
* BATCH_SIZE = 2
* 有在Encoder中載入用**noisy-student**方法預訓練的模型參數
* **loss function: dice_loss**

### 讀取函式庫、超參數設定

In [1]:
### 讀取函式庫 ###
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time

In [2]:
import tensorflow as tf
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


In [3]:
dataset_dir = 'DATA_0811_2\\WW600WL100'
x_train_path = 'DATA_0811_2\\WW600WL100\\train\\images'
y_train_path = 'DATA_0811_2\\WW600WL100\\train\\annotations_tumor as kidney'
checkpoints_path = 'DATA_0811_2\WW600WL100\\Model_EfficientNetB4-Unet_diceLoss_1004'
model_name = 'EfficientNetB4-Unet'
start_from_latest_checkpoints = False

INPUT_SIZE = 512 # input size equal to output size
CLASS_NUM = 2 # background and roi(kidney)

# initialize the number of epochs and batch size
EPOCHS = 100
BATCH_SIZE = 2

IMAGE_ORDERING = 'channels_last' # 不要亂改這個參數

### 用生成器(generator)生成訓練圖片
使用generator的好處在於，模型在訓練(或驗證)的過程中會依據批量大小生成圖片，不需要在程式中預先準備好資料集(通常資料集都會大到無法一口氣塞入記憶體之中)，減少記憶體的使用空間。另外，使用ImageDataGenerator可以對生成的圖片實施「資料增強(Data Augmentation)」，增加模型的泛化能力。

In [4]:
### 取得資料集中所有的檔案路徑(包含CT影像和其對應標記的路徑) ###
def get_pairs_from_paths(images_path, segs_path, ignore_non_matching=False):
    """ Find all the images from the images_path directory and
        the segmentation images from the segs_path directory
        while checking integrity of data """

    ACCEPTABLE_IMAGE_FORMATS = [".jpg", ".jpeg", ".png", ".bmp"]
    ACCEPTABLE_SEGMENTATION_FORMATS = [".png", ".bmp"]

    image_files = []
    segmentation_files = {}

    for dir_entry in os.listdir(images_path):
        if os.path.isfile(os.path.join(images_path, dir_entry)) and \
                os.path.splitext(dir_entry)[1] in ACCEPTABLE_IMAGE_FORMATS:
            file_name, file_extension = os.path.splitext(dir_entry)
            image_files.append((file_name, file_extension,
                                os.path.join(images_path, dir_entry)))

    for dir_entry in os.listdir(segs_path):
        if os.path.isfile(os.path.join(segs_path, dir_entry)) and \
           os.path.splitext(dir_entry)[1] in ACCEPTABLE_SEGMENTATION_FORMATS:
            file_name, file_extension = os.path.splitext(dir_entry)
            full_dir_entry = os.path.join(segs_path, dir_entry)
            if file_name in segmentation_files:
                raise DataLoaderError("Segmentation file with filename {0}"
                                      " already exists and is ambiguous to"
                                      " resolve with path {1}."
                                      " Please remove or rename the latter."
                                      .format(file_name, full_dir_entry))

            segmentation_files[file_name] = (file_extension, full_dir_entry)

    return_value = []
    # Match the images and segmentations
    for image_file, _, image_full_path in image_files:
        if image_file in segmentation_files:
            return_value.append((image_full_path,
                                segmentation_files[image_file][1]))
        elif ignore_non_matching:
            continue
        else:
            # Error out
            raise DataLoaderError("No corresponding segmentation "
                                  "found for image {0}."
                                  .format(image_full_path))

    return return_value

### 將CT影像的陣列轉成適合模型輸入的形式(維度轉換 + 標準化) ###
def get_image_array(image_input,
                    width, height,
                    imgNorm="sub_mean", ordering='channels_first'):
    """ Load image array from input """

    if type(image_input) is np.ndarray:
        # It is already an array, use it as it is
        img = image_input
    elif isinstance(image_input, six.string_types):
        if not os.path.isfile(image_input):
            raise DataLoaderError("get_image_array: path {0} doesn't exist"
                                  .format(image_input))
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_image_array: Can't process input type {0}"
                              .format(str(type(image_input))))

    if imgNorm == "sub_and_divide": # 除以127.5，然後減 1
        img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    elif imgNorm == "sub_mean": # 減去ImageNet的平均BGR
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        img = img[:, :, ::-1]
    elif imgNorm == "divide": # 除以255
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img = img/255.0

    if ordering == 'channels_first':
        img = np.rollaxis(img, 2, 0)
    return img

### 將標記資料的陣列轉成適合模型輸入的形式(維度轉換) ###
def get_segmentation_array(image_input, nClasses,
                           width, height, no_reshape=False):
    """ Load segmentation array from input """

    seg_labels = np.zeros((height, width, nClasses))

    if type(image_input) is np.ndarray:
        # It is already an array, use it as it is
        img = image_input
    elif isinstance(image_input, six.string_types):
        if not os.path.isfile(image_input):
            raise DataLoaderError("get_segmentation_array: "
                                  "path {0} doesn't exist".format(image_input))
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_segmentation_array: "
                              "Can't process input type {0}"
                              .format(str(type(image_input))))

    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_NEAREST)
    img = img[:, :, 0]

    for c in range(nClasses):
        seg_labels[:, :, c] = (img == c).astype(int)

    if not no_reshape:
        seg_labels = np.reshape(seg_labels, (width*height, nClasses))

    return seg_labels

import itertools
import random
from keras.preprocessing.image import ImageDataGenerator

def image_segmentation_generator(images_path, segs_path, batch_size,
                                 n_classes, input_height, input_width,
                                 output_height, output_width, do_augment=False):

    img_seg_pairs = get_pairs_from_paths(images_path, segs_path) 
        # 取得資料集中所有的檔案路徑(包含CT影像和其對應標記的路徑)
    random.shuffle(img_seg_pairs) # 打散檔案路徑
    zipped = itertools.cycle(img_seg_pairs) 
        # 將檔案路徑用循環迭代器(iterator)封裝；範例：cycle('ABCD') --> A B C D A B C D ...
#     counter = 0

    while True:
        # 如果模型訓練完一輪訓練集中所有的資料，就重新打散檔案路徑
#         if counter >= len(img_seg_pairs):
#             counter = 0
#             random.shuffle(img_seg_pairs)
#             zipped = itertools.cycle(img_seg_pairs)             
        
        X = []
        Y = []
        for _ in range(batch_size): # batch_size多大，就取得多少份資料
#             counter += 1
            
            im, seg = next(zipped) # 取得CT影像和其對應標記的路徑

            im = cv2.imread(im, 1) # 1 = cv2.IMREAD_COLOR (讀取彩色圖片)
            seg = cv2.imread(seg, 1)
            
            if do_augment:
                ### Example of transforming images and masks together. ###
                # we create two instances with the same arguments
                data_gen_args = dict(featurewise_center=False, # 範例程式碼為True，但這裡我只是要把一張圖片變成是增強後的型態
                         featurewise_std_normalization=False, # 範例程式碼為True，但這裡我只是要把一張圖片變成是增強後的型態
                         #rotation_range=30, # 範例程式碼為90
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.2)
                image_datagen = ImageDataGenerator(**data_gen_args)
                mask_datagen = ImageDataGenerator(**data_gen_args)
                # Provide the same seed and keyword arguments to the fit and flow methods
                seed = random.randint(0,10000)
                im_itr = image_datagen.flow(im.reshape(1, 512, 512, 3), batch_size=1, seed=seed)  
                seg_itr = mask_datagen.flow(seg.reshape(1, 512, 512, 3), batch_size=1, seed=seed)  
                im = next(im_itr).reshape(512, 512, 3)
                seg = next(seg_itr).reshape(512, 512, 3)
            
            # 將CT影像和其對應的標記轉換成模型輸入的形式
            X.append(get_image_array(im, input_width,
                                     input_height, imgNorm="sub_mean", ordering=IMAGE_ORDERING)) 
                # imgNorm預設為sub_mean"，但這裡我改用圖片最常實施的正規化方法(同除以255)
            Y.append(get_segmentation_array(
                seg, n_classes, output_width, output_height))

        yield np.array(X), np.array(Y)

Using TensorFlow backend.


In [5]:
# 建立訓練資料的生成器
train_gen = image_segmentation_generator(images_path = x_train_path, segs_path = y_train_path, batch_size = BATCH_SIZE, 
                                         n_classes = CLASS_NUM, input_height = INPUT_SIZE, input_width = INPUT_SIZE, 
                                         output_height = INPUT_SIZE, output_width = INPUT_SIZE, do_augment=False)

### 建立模型

In [6]:
import keras
from keras.models import *
from keras.layers import *
from keras import layers

#### Eecoder - EfficientNetB4

In [7]:
import efficientnet.keras as efn 

model_ = efn.EfficientNetB4(
    input_shape=(INPUT_SIZE,INPUT_SIZE,3), 
    weights='noisy-student'
)

model_.summary()

Model: "efficientnet-b4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 48) 0           stem_bn[0][0]                    
____________________________________________________________________________________

In [8]:
### B4 ###
img_input = model_.get_layer('input_1').output # (None, 512, 512, 3)
f1 = model_.get_layer('block1b_add').output # (None, 256, 256, 24)
f2 = model_.get_layer('block2d_add').output # (None, 128, 128, 32)
f3 = model_.get_layer('block3d_add').output # (None, 64, 64, 56)
f4 = model_.get_layer('block4f_add').output # (None, 32, 32, 112)
f5 = model_.get_layer('block5f_add').output # (None, 32, 32, 160)
f6 = model_.get_layer('block6h_add').output # (None, 16, 16, 272)
f7 = model_.get_layer('block7b_add').output # (None, 16, 16, 448)

#### Decoder

In [9]:
if IMAGE_ORDERING == 'channels_first':
    MERGE_AXIS = 1
elif IMAGE_ORDERING == 'channels_last':
    MERGE_AXIS = -1

o = f7

#o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f6], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f5], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

#o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f4], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(128, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f3], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f2], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f1], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = Conv2D(CLASS_NUM, (3, 3), padding='same', data_format=IMAGE_ORDERING)(o) # 輸出層

### 模仿 keras_segmentation 在輸出層之後額外添加這幾層 ###
if IMAGE_ORDERING == 'channels_first':
    o = (Reshape((CLASS_NUM, INPUT_SIZE * INPUT_SIZE)))(o)
    o = (Permute((2, 1)))(o)
elif IMAGE_ORDERING == 'channels_last':
    o = (Reshape((INPUT_SIZE * INPUT_SIZE, CLASS_NUM)))(o)
o = (Activation('softmax'))(o)
###

In [10]:
model = Model(img_input, o)

#### 查看模型架構

B4：
* Total params: 22,355,674
* Trainable params: 22,231,882
* Non-trainable params: 123,792

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 48) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

### 自定義損失函數

參考來源：
* Dice loss：https://gist.github.com/wassname/7793e2058c5c9dacb5212c0ac0b18a8a

In [12]:
from keras.losses import categorical_crossentropy

def dice_coef(y_true, y_pred, smooth = 1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    
    # A和B取平方是為了多類別的預測，如果是二元分類的話，就跟原來的Dice公式一樣
    
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis = -1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true), -1) + K.sum(K.square(y_pred), -1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# def sum_of_weighted_crossentropy_and_dice_loss(dice_weight = 0.5):
#     def loss(y_true, y_pred):
#         return (1 - dice_weight) * categorical_crossentropy(y_true, y_pred) + dice_weight * (1 - dice_coef(y_true, y_pred))
#     return loss

### 編譯模型

In [13]:
from keras.optimizers import Adam, Adadelta
model.compile(loss = dice_coef_loss,
              #loss = 'categorical_crossentropy',
              optimizer = Adadelta(lr = 1.0),
              metrics = ['accuracy'])

import keras.backend as K
print(K.eval(model.optimizer.lr)) # 確保是1.0

1.0


### 設定回調函式(callbacks)
keras_segmentation自定義的回調函式好像缺了什麼，直接用在這份程式碼中並不會正常運作，因此我直接用keras.callbacks.ModelCheckpoint定期儲存模型參數。

In [14]:
from keras.callbacks import ModelCheckpoint
callbacks = [ModelCheckpoint(os.path.join(checkpoints_path, model_name + '_{epoch}.h5'), 
                             save_weights_only=True, 
                             period=5)] # period=5：每5個Epoch才會儲存一次參數

### 訓練模型

In [15]:
st = time.time()
print(f'########## 即將用該資料集進行訓練：{dataset_dir} ##########')

if start_from_latest_checkpoints:
    try:
        model.load_weights(
            os.path.join(
                checkpoints_path, 
                model_name + '_' + str(max([int(i.split('_')[-1][:-3]) for i in os.listdir(checkpoints_path)])) + '.h5'
            )
        )
    except:
        print('Model weights not found!')

if not os.path.exists(checkpoints_path):
    os.makedirs(checkpoints_path)
    print('-----建立新資料夾：' + checkpoints_path + '-----') 


model.fit_generator(
    train_gen,
    steps_per_epoch = 512, 
    #steps_per_epoch = len(x_train) // BATCH_SIZE,
    epochs = EPOCHS,
    verbose = 1,
    callbacks=callbacks
)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

########## 即將用該資料集進行訓練：DATA_0811_2\WW600WL100 ##########
-----建立新資料夾：DATA_0811_2\WW600WL100\Model_EfficientNetB4-Unet_diceLoss_1004-----
Epoch 1/100
512/512 [==============================] - 254s 496ms/step - loss: 0.0298 - accuracy: 0.9551
Epoch 2/100
512/512 [==============================] - 213s 417ms/step - loss: 0.0056 - accuracy: 0.9916
Epoch 3/100
512/512 [==============================] - 213s 416ms/step - loss: 0.0057 - accuracy: 0.9915
Epoch 4/100
512/512 [==============================] - 214s 417ms/step - loss: 0.0056 - accuracy: 0.9914
Epoch 5/100
512/512 [==============================] - 214s 417ms/step - loss: 0.0059 - accuracy: 0.9908
Epoch 6/100
512/512 [==============================] - 213s 416ms/step - loss: 0.0059 - accuracy: 0.9905
Epoch 7/100
512/512 [==============================] - 213s 416ms/step - loss: 0.0059 - accuracy: 0.9902
Epoch 8/100
512/512 [==============================] - 215s 420ms/step - loss: 0.0047 - accuracy: 0.9921
Epoch 9/100
512/512 [==

512/512 [==============================] - 213s 417ms/step - loss: 3.4439e-04 - accuracy: 0.9994
Epoch 76/100
512/512 [==============================] - 214s 417ms/step - loss: 3.5682e-04 - accuracy: 0.9994
Epoch 77/100
512/512 [==============================] - 213s 416ms/step - loss: 3.5414e-04 - accuracy: 0.9994
Epoch 78/100
512/512 [==============================] - 213s 415ms/step - loss: 3.2063e-04 - accuracy: 0.9994
Epoch 79/100
512/512 [==============================] - 213s 415ms/step - loss: 3.4508e-04 - accuracy: 0.9994
Epoch 80/100
512/512 [==============================] - 213s 416ms/step - loss: 3.2129e-04 - accuracy: 0.9994
Epoch 81/100
512/512 [==============================] - 213s 416ms/step - loss: 3.2649e-04 - accuracy: 0.9994
Epoch 82/100
512/512 [==============================] - 213s 415ms/step - loss: 3.5795e-04 - accuracy: 0.9994
Epoch 83/100
512/512 [==============================] - 214s 417ms/step - loss: 3.0827e-04 - accuracy: 0.9995
Epoch 84/100
512/512 [=

### 模型預測結果(測試集)

In [16]:
### 載入模型參數 ###
# model.load_weights(os.path.join(checkpoints_path, model_name + '_' + EPOCHS + '.h5'))

In [17]:
from evaluate import *
preds = predict_from_folder(model, 'DATA_0811_2\\WW600WL100\\test\\images', INPUT_SIZE, 2)

100%|████████████████████████████████████████████████████████████████████████████| 12656/12656 [12:07<00:00, 17.40it/s]


In [18]:
st = time.time()

result = evaluate_model(
    image_dir = os.path.join(dataset_dir, 'test', 'images'), 
    label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
    checkpoints_path = None,
    calculate_predicting_indicators = True,
    output_predicted_result = False, 
    segment_out_predicted_region_from_original_images = False, 
    roi_description = 'tumor as kidney', 
    preds = preds)

print('測試集預測結果：')
print(f'average Dice score per case of kidney: {result[0]: .4f}')
print(f'average recall of kidney: {result[1]: .4f}')
print(f'average precision of kidney: {result[2]: .4f}')
print(f'global dice score of kidney: {result[3]: .4f}')
print('')
print('混淆矩陣：')
print(f'True Positive: {result[-4]}')
print(f'False Positive: {result[-3]}')
print(f'False Negative: {result[-2]}')
print(f'True Negative: {result[-1]}')

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

----------預測結果資料型態轉換(第1批資料)----------
----------開始計算各項預測指標(第1批資料)----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
total case number: 12656
測試集預測結果：
average Dice score per case of kidney:  0.9480
average recall of kidney:  0.9645
average precision of kidney:  0.9362
global dice score of kidney:  0.9460

混淆矩陣：
True Positive: 4005
False Positive: 626
False Negative: 114
True Negative: 7911
花費時間(秒)：702.5175879001617


In [19]:
f = open("DATA_0811_2\\patient indices of testing set - KiTS.txt", "r")
test_patient_idx = f.read().splitlines()
f.close()

dice_score_list = result[5]
print('測試集各病患的 Dice score:')
for idx, i in enumerate(dice_score_list):
    print(f'case{test_patient_idx[idx]}: {i: .4f}')

測試集各病患的 Dice score:
case150:  0.9601
case151:  0.8938
case152:  0.9581
case153:  0.9747
case154:  0.9474
case155:  0.9627
case156:  0.9714
case157:  0.9589
case158:  0.9809
case159:  0.9549
case160:  0.9411
case161:  0.9661
case162:  0.9486
case163:  0.9763
case164:  0.9675
case165:  0.9733
case166:  0.9581
case167:  0.9497
case168:  0.9599
case169:  0.9705
case170:  0.9514
case171:  0.9549
case172:  0.9400
case173:  0.9516
case174:  0.9646
case175:  0.9541
case176:  0.9144
case177:  0.9045
case178:  0.9166
case179:  0.9711
case181:  0.9720
case182:  0.9723
case183:  0.9446
case184:  0.9207
case185:  0.9536
case186:  0.9758
case187:  0.9021
case188:  0.9377
case189:  0.9102
case190:  0.9667
case191:  0.9695
case192:  0.9541
case193:  0.9699
case194:  0.9476
case195:  0.9649
case196:  0.9776
case197:  0.5862
case198:  0.9489
case199:  0.9567
case200:  0.9536
case201:  0.9798
case202:  0.9658
case204:  0.9776
case205:  0.9425
case206:  0.9447
case207:  0.9598
case208:  0.9570
case209:  0